<center> 
  <h1> Mind The Duality Gap : Safer Rules For The Lasso </h1> 
</center>

In [4]:
!pip install python-intervals

In [0]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import linalg
from numpy.random import multivariate_normal
from scipy.linalg.special_matrices import toeplitz 
from numpy.random import randn
from intervals import Interval, inf
from scipy.optimize import minimize

One chooses Coordinate Descent as iterative solver. <br/>
The idea of coordinate descent is to decompose a large optimisation problem into
a sequence of one-dimensional optimisation problems. 
Coordinate descent methods have become unavoidable in machine learning because
they are very efficient for key problems, namely Lasso, Logistic Regression and
Support Vector Machines. <br/>
Moreover, the decomposition into small subproblems means that only a small part 
of the data is processed at each iteration and this makes coordinate descent
easily scalable to high dimensions. 
The idea of coordinate gradient descent is to perform one iteration of gradient
in the 1-dimensional problem :
$$
min_{z \in X_{i}} f(x_{k}^{(1)}, \ldots, x_{k}^{(l-1)}, z, x_{k}^{(l+1)}, \ldots, x_{k}^{(n)})
$$
instead of solving it completely. In general it reduces drastically the cost 
of each iteration while keeping the same convergence behaviour.

## Data Simulation

In [0]:
from numpy.random import multivariate_normal
from scipy.linalg.special_matrices import toeplitz
from numpy.random import randn

def simu(beta, n_samples=1000, corr=0.5, for_logreg=False):
    n_features=len(beta)
    cov = toeplitz(corr ** np.arange(0, n_features))

    # Features Matrix
    X = multivariate_normal(np.zeros(n_features), cov, size=n_samples)

    # Target labels vector with noise
    y = X.dot(beta) + randn(n_samples)

    if for_logreg:
        y = np.sign(y)

    return X, y

## Test Cell

In [0]:
np.random.seed(0)
n_features = 100
beta = np.random.randn(n_features)

X, y = simu(beta, n_samples=1000, corr=0.5, for_logreg=False)

# print("Features matrix X : \n", X)
# print("Target vector y : \n", y)

## Coordinate Descent Algorithm

In [0]:
def cyclic_coordinate_descent(X, y, n_iter=10):
    """Solver : cyclic coordinate descent 

    Parameters
    ----------

    X: numpy.ndarray, shape (n_samples, n_features)
       features matrix

    y: numpy.array, shape (n_samples, )
       target labels vector

    n_iter: int, default = 10
            number of iterations  

    Returns
    -------

    beta: numpy.array, shape(n_features,)
          parameters vector

    all_objs: numpy.array, shape(n_features)
              residuals vector

    """

    # Initialisation of the parameters

    n_samples, n_features = X.shape
    all_objs = []

    beta = np.zeros(n_features)
    residuals = y - X.dot(beta)

    # Computation of the lipschitz constants vector 

    lips_const = np.linalg.norm(X, axis=0)**2

    # Iterations of the algorithm
    for k in range(n_iter):

        # One cyclicly updates the i^{th} coordinate corresponding to the rest 
        # in the Euclidean division by the number of features 
        # This allows to always selecting an index between 1 and n_features
        i = k % n_features + 1

        old_beta_i = beta[i].copy()
        step = 1/lips_const[i] 
        reg = old_beta_i * X[:,i]
        grad = (X[:,i].T).dot(residuals + reg)

        # Update of the parameters
        beta[i] += step*grad 

        # Update of the residuals
        residuals += np.dot(X[:,i], old_beta_i - beta[i])

        if k % n_features == 0:
            # If k % n_features == 0 then we have updated all the coordinates
            # This means that we have performed a whole cycle 
            # One computes the objective function 
            all_objs.append((residuals**2).sum()/2.)

    return beta, np.array(all_objs)       


## Test Cell

In [9]:
%%time
beta_hat_cyclic_cd, objs_cyclic_cd = cyclic_coordinate_descent(X, y, n_iter=10)

# print("Beta hat cyclic coordinate descent : \n", beta_hat_cyclic_cd)
# print("Objective function for the optimum parameters vector beta hat coordinate descent : \n", objs_cyclic_cd)

CPU times: user 2.29 ms, sys: 80 µs, total: 2.37 ms
Wall time: 1.94 ms


In [15]:
%%time
beta_hat_ols, objs_ols,_,_ = np.linalg.lstsq(X, y)

# print("Beta hat OLS : \n", beta_hat_ols)
# print("Objective function for the optimum parameters vector beta hat ols ! \n", objs_ols)

CPU times: user 12.1 ms, sys: 12 ms, total: 24.1 ms
Wall time: 21.3 ms


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  """Entry point for launching an IPython kernel.


**Results** <br/>

We can observe that the cyclic coordinate descent is much more efficient than the ordinary least square solver, especially regarding its time computation. <br/>
This phenomenon can be explained by the fact that the coordinate cyclic descent is based onto an update of the parameters vector coordinate by coordinate whereas the ordinary least squares relies on an update of all the coordinates of the parameters vector at each iteration what is very expensive. 

## Coordinate Descent With Gap Safe Rules

In [0]:
# Equation 11
def compute_theta_k(X, y, beta_k, lmbda):
  """Iterative computation of the dual optimal solution theta
     Dynamic Safe Rules

  Parameters
  ----------

  X: numpy.ndarray, shape = (n_samples, n_features)
     features matrix

  y: numpy.array, shape = (n_features, )
     target labels vector

  lmbda: numpy.array, shape = (n_iter, )
         regularization parameters vector

  beta_k: numpy.array, shape = (n_features, )
          primal optimal parameters vector

  Returns
  -------

  theta_k: float 
           dual optimal parameters vector

  """

  # Initialization of the parameters 
      # Residuals vector 
  rho_k = y - np.dot(X, beta_k)
      # Proportionality constant 
  a_k = np.dot(y.T, rho_k)/(lmbda*np.linalg.norm(rho_k)**2)
  b_k = -1/np.linalg.norm(np.dot(X.T, rho_k), np.inf)
  c_k = 1/np.linalg.norm(np.dot(X.T, rho_k), np.inf)

  alpha_k = min(max(a_k, b_k), c_k)
      # Dual optimal parameters vector
  theta_k = alpha_k * rho_k

  return theta_k
  

## Test Cell

In [17]:
%%time
lmbda = 0.1
theta_k = compute_theta_k(X, y, beta_k=beta, lmbda=lmbda)

# print("Dual optimal paramters vector at iteration k theta_k : \n", theta_k)

CPU times: user 2.22 ms, sys: 1.61 ms, total: 3.83 ms
Wall time: 3.97 ms


In [0]:
def radius_eq20(beta, theta, lmbda_t, lmbda_t_1, r_lmbda_t_1):
  """
  Parameters
  ----------

  beta: numpy.array, shape = (n_features, )
        primal optimal parameters vector

  theta: numpy.array, shape = (n_features, )
         dual optimal parameters vector

  lmbda_t: float
           regularization parameter at iteration t

  lmbda_t_1: float
             regularization parameter at iteration t-1
  
  r_lmbda_t_1: float
               radius related to the regularization parameter lmbda_t_1

  Returns
  -------
  r_lmbda_t: float
             radius related to the regularization parameter lmbda_t 
  """

  r_square_lmbda_t = (lmbda_t_1/lmbda_t)*r_lmbda_t_1**2 + (1 - lmbda_t/lmbda_t_1)*np.linalg.norm((X @ beta - y)/lmbda_t)**2 - (lmbda_t_1/lmbda_t - 1)*np.linalg.norm(theta)**2
  r_lmbda_t = np.sqrt(r_square_lmbda_t)

  return r_lmbda_t

In [0]:
# Equation 18 : Gap Safe Sphere

def gap_safe_sphere(theta_k, r_lmbda):
  """
  Parameters
  ----------

  theta_k: numpy.array, shape = (n_features,)
           dual optimal parameters vector
  
  r_lmbda: float
           radius related to the regularization parameter lmbda

  Returns
  -------

  C_k: interval
       sphere of center theta_k and of radius r_lmbda
  """

  inf_bound = theta_k - r_lmbda
  sup_bound = theta_k + r_lmbda
  C_k = interval[inf_bound, sup_bound]
  
  return C_k

In [0]:
def R_primal(X, y, beta, lmbda):
  """
  Parameters
  ----------
  
  X: numpy.ndarray, shape=(n_samples, n_features)
     features matrix

  y: numpy.array, shape=(n_samples, )
     target labels vector 

  beta: numpy.array, shape=(n_features, )
        primal optimal parameters vector

  lmbda: float
         regularization parameter

  Returns
  -------

  R_hat_lmbda: float
               primal radius of the dome
  """

  R_hat_lmbda = (1/lmbda)*np.max(np.linalg.norm(y)**2 - np.linalg.norm(np.dot(X, beta) - y)**2 - 2*lmbda*np.linalg.norm(beta, 1), 0)**(1/2)

  return R_hat_lmbda

## Test Cell

In [23]:
R_hat_lmbda = R_primal(X, y, beta, lmbda)

print("R primal (R_hat_lmbda) : \n", R_hat_lmbda)

R primal (R_hat_lmbda) : 
 3464.8358952601366


In [0]:
def R_dual(y, theta, lmbda):
  """
  Parameters
  ----------
  
  y: numpy.array, shape=(n_samples, ) 
     target labels vector 

  theta: numpy.array, shape=(n_features, )
        dual optimal parameters vector

  lmbda: float
         regularization parameter

  Returns
  -------

  R_inv_hat_lmbda: float
                   dual radius of the dome
  """

  R_inv_hat_lmbda = np.linalg.norm(theta - y/lmbda)

  return R_inv_hat_lmbda

## Test Cell

In [26]:
R_inv_hat_lmbda = R_dual(y, theta_k, lmbda)

print("R dual (R_inv_hat_lmbda) : \n", R_inv_hat_lmbda)

R dual (R_inv_hat_lmbda) : 
 3479.259836114735


In [0]:
def radius_thm2(R_hat_lmbda, R_inv_hat_lmbda):
  """Compute the radius of the safe sphere region

  Parameters
  ----------

  R_hat_lmbda: float 
               primal radius of the safe dome region

  R_inv_hat_lmbda: float
                   dual radius of the safe dome region

  Returns
  -------
  r_lmbda: float
           radius of the safe sphere region
  """

  r_lmbda = np.sqrt(R_inv_hat_lmbda**2 - R_hat_lmbda**2)

  return r_lmbda

## Test Cell

In [29]:
r_lmbda = radius_thm2(R_hat_lmbda, R_inv_hat_lmbda)

print("Radius of the safe sphere region r_lmbda : \n", r_lmbda)

Radius of the safe sphere region r_lmbda : 
 316.4825842254526


In [0]:
# Equation 19 : Gap Safe Dome
def gap_safe_dome(y, lmbda, theta_k, beta_k, R_hat_lmbda, R_inv_hat_lmbda):
  """
  Parameters
  ---------

  y: np.array, shape = (n_samples, )

  lmbda: float 
         regularization parameter
  
  theta_k: np.array, shape = (n_features, )
           dual optimal parameters vector

  beta_k: np.array, shape = (n_features, )
          primal optimal parameters vector
  
  R_hat_lmbda: float
               primal radius of the dome

  R_inv_hat_lmbda: float
                   dual radius of the dome
  

  Returns
  -------

  C_k: interval
       gape safe dome

  """

  c_k = ((y/lmbda) + theta_k)/2
  r_k = R_inv_hat_lmbda/2
  sphere_k = interval[c_k - r_k, c_k + r_k]

  alpha_k = 2*(R_hat_lmbda/R_inv_hat_lmbda)**2 - 1
  w_k = (theta_k - y/lmbda)/np.linalg.norm(theta_k - y/lmbda)

  margin_k = - alpha_k*r_k*w_k

  # c_k - alpah_k*r_k*w_k is the projection of the ball center c on the hyperplane
  # c = ball center
  # r = ball radius
  # oriented hyperplane with unit normal vector w and parameter alpha 
  # such that c - alpha*r*w is the projection of c on the hyperplane

  C_k = sphere_k.intersection(margin_k) # problème d'intersection avec l'hyperplane

  return C_k

In [0]:
def sigma_support_function(x_j, theta):
  """Support Function

  Parameters
  ----------

  x_j: np.array, shape = (n_samples, )
       vector of samples for a given feature j

  theta: np.array, shape = (n_samples, )
         vector of parameters 

  Returns
  -------

  sigma: float
         scalar product <x, theta> between the j^th feature X[:,j] 
         and the vector of parameters theta
  """
  sigma = -np.dot(x_j.T, theta)
  
  return sigma

## Test Cell

In [0]:
x_1 = X[:,1]
theta = np.random.randn(x_1.shape[0])
sigma = sigma_support_function(x_1, theta)

In [0]:
def sigma_C(x_j, theta):
  """Support Function

  Parameters
  ----------

  x_j: np.array, shape = (n_samples, )
       vector of samples for a given feature j  

  theta: np.array, shape = (n_samples, )
         vector of parameters

  Returns
  -------

  support_function: float
                    maximum value of the scalar product between
                    the j^th feature X[:,j] and the vector of parameters theta
                    over the convex set C 
  """
  support_function = minimize(sigma_support_function, x_j, theta, method='nelder-mead', options={'xatol': 1e-8, 'disp': True})
  support_function = -support_function
  return support_function

## Test Cell

In [88]:
%%time
x_1 = X[:,1]
theta = np.random.randn(x_1.shape[0])
sigma = sigma_support_function(x_1, theta)
# support_function = sigma_C(x_1, theta)
print(sigma)

30.922503744184503
CPU times: user 201 µs, sys: 985 µs, total: 1.19 ms
Wall time: 2.68 ms


In [0]:
def mu_C(x_j, theta):
  """
  Parameters
  ----------
  x_j: np.array, shape = (n_samples, )
       feature x_j 

  Returns
  -------
  mu: float 
      maximum between two sigma_C(x_j) and sigma_C(-x_j)
  """

  # mu = max(sigma_C(x_j, theta), sigma_C(-x_j, theta))
  mu = max(sigma_support_function(x_j, theta), sigma_support_function(-x_j, theta))

  return mu

## Test Cell

In [93]:
mu = mu_C(x_1, theta)

print("mu C : \n", mu) # Problème d'optimiseur 

mu C : 
 30.922503744184503


In [0]:
def mu_B(x_j, c, r):
  """Function mu applied to the sphere of center c and radius r 
     for the j^th feature X[:,j]

  Parameters
  ----------
  x_j: numpy.array, shape=(n_samples, )
       j^th feature X[:,j]

  c: float
     center of the sphere
     
  r: float
     radius of the sphere

  Returns
  -------

  mu: float
      maximum value between the scalar products of theta and x_j
      and theta and -x_j
  """

  mu = abs(np.dot(x_j.T, c)) + r*np.linalg.norm(x_j)

  return mu

## Test Cell

In [139]:
x_1 = X[:,1]
c = np.random.randn(x_1.shape[0])
r = 1
mu = mu_B(x_1, c, r)

print("mu value : \n", mu)

mu value : 
 63.81002868883159


In [0]:
# Equation 7 : Active Set
def active_set_vs_zero_set(X):
  """
  Parameters
  ----------
  X: numpy.ndarray, shape = (n_samples, n_features)
     features matrix 

  Returns
  -------
  A_C: numpy.array, shape = (n_idx_active_features, )
       active set : contains the indices of the relevant features
  
  Z_C: numpy.array, shape = (n_idx_zero_features, )
       zero set : contains the indices of the irrelevant features
  """

  theta = np.random.randn(x_1.shape[0]) # Initialisation ? 
  A_C = []
  Z_C = []
  p = X.shape[1]
  for j in range(p):
    mu = mu_C(X[:,j], theta)
    if mu >= 1:
      A_C.append(j)
    else:
      Z_C.append(j)

  return A_C, Z_C

## Test Cell

In [141]:
A_C, Z_C = active_set_vs_zero_set(X)

print("Active Set : \n", A_C)
print("Zero Set : \n", Z_C)

Active Set : 
 [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99]
Zero Set : 
 [27]


In [0]:
def sign(x):
  """
  Parameters
  ----------
  x: float
     
  Returns
  -------
  s: sign int
     (-1) if x < 0,, (+1) if x > 0, 0 if x = 0
  """

  if x > 0:
    s = 1
  elif x < 0:
    s = -1
  else:
    s = 0

  return s

## Test Cell

In [0]:
x = 2
s = sign(x)

In [0]:
def soft_thresholding(u,x):
  """

  Parameters
  ----------
  u: float
     threshold

  x: float

  Returns
  -------
  ST: float
      0 between -u and +u or slope of the straight line x - u otherwise 
      
  """

  ST = sign(x)*np.max(np.abs(x) - u, 0)


  return ST

## Test Cell

In [0]:
u = 1
ST = soft_thresholding(u,x)

In [0]:
# Equation 1 : Primal Problem 
def primal_pb(X, y, beta, lmbda):
  """
  Parameters
  ----------
  X: numpy.ndarray, shape = (n_samples, n_features)
     features matrix

  y: numpy.array, shape = (n_samples, )
     target labels vector

  beta: numpy.array, shape = (n_features, )
        initial vector of primal parameters

  lmbda: float
         regularization parameter

  Returns
  -------

  beta_hat_lmbda: numpy.array shape = (n_features, )
                  primal optimal parameters vector
  """

  P_lmbda = (1/2)*np.linalg.norm(np.dot(X, beta) - y, 2)**2 + lmbda*np.linalg.norm(beta, 1)

  return beta_hat_lmbda

In [0]:
def dual_pb(y, theta, lmbda):
  """
  Parameters
  ----------
  y: numpy.array, shape = (n_features, )

  theta: numpy.array, shape = (n_features, )
         initial vector of dual parameters

  lmbda: float
         regularization parameter

  Returns
  -------

  theta_hat_lmbda: numpy.array, shape = (n_features, )
                   dual optimal parameters vector

  """

  D_lmbda = (1/2)*np.linalg.norm(y, ord=2)**2 - ((lmbda**2)/2)*np.linalg.norm(theta - y/lmbda, ord=2)**2
  
  return theta_hat_lmbda

## Maximization of the dual problem

In [10]:
def cyclic_coordinate_descent(X, y, lmbda, n_iter=10):
    """Solver : cyclic coordinate descent 

    Parameters
    ----------

    X: numpy.ndarray, shape (n_samples, n_features)
       features matrix

    y: numpy.array, shape (n_samples, )
       target labels vector

    lmbda: float
           regularization parameter

    n_iter: int, default = 10
            number of iterations  

    Returns
    -------

    theta: numpy.array, shape(n_features,)
           dual parameters vector

    all_objs: numpy.array, shape(n_features)
              residuals vector

    """

    # Initialisation of the parameters

    n_samples, n_features = X.shape
    all_objs = []

    theta = np.zeros(n_features)
    residuals = theta - y/lmbda

    # Computation of the lipschitz constants vector 

    lips_const = np.linalg.norm(X, axis=0)**2

    # Iterations of the algorithm
    for k in range(n_iter):

        # One cyclicly updates the i^{th} coordinate corresponding to the rest 
        # in the Euclidean division by the number of features 
        # This allows to always selecting an index between 1 and n_features
        i = k % n_features + 1

        old_theta_i = theta[i].copy()
        if abs(np.dot(X[:,i].T, y/lmbda)) <= 1:
          step = 1/lips_const[i] 
          reg = old_theta_i * X[:,i]
          grad = (X[:,i].T).dot(residuals + reg)

          # Update of the parameters
          theta[i] += step*grad 

        # Update of the residuals
        residuals += np.dot(X[:,i], old_beta_i - beta[i])

        if k % n_features == 0:
            # If k % n_features == 0 then we have updated all the coordinates
            # This means that we have performed a whole cycle 
            # One computes the objective function 
            all_objs.append((residuals**2).sum()/2.)

    return beta, np.array(all_objs)       


IndentationError: ignored

In [0]:
def duality_gap(beta_hat_lmbda, theta_hat_lmbda):
  """
  Parameters
  ----------

  beta_hat_lmbda: numpy.array shape = (n_features, )
                  primal optimal parameters vector

  theta_hat_lmbda: numpy.array, shape = (n_features, )
                   dual optimal parameters vector

  Returns
  -------
  G_lmbda: float
           duality gap between the primal optimal and the dual optimal

  """

  G_lmbda = beta_hat_lmbda - theta_hat_lmbda

  return G_lmbda


In [0]:
def cd_with_gap_safe_rules(X, y, epsilon, K, f, lmbda, T, lmbda_max):
  """Coordinate descent with gap safe rules 

  Parameters
  ----------
  X: numpy.ndarray, shape = (n_samples, n_features)
     features matrix

  y: numpy.array, shape = (n_features, )
     target labels vector

  epsilon: float 
           accuracy

  T: int
     number of epochs

  K: int
     number of iterations

  f: int
     frequency

  lmbda: numpy.array, shape = (T-1, )
          regularization parameters vector

  Returns
  -------

  beta_lmbda_t: numpy.array, shape = (n_features, )
                primal optimal parameters vector 
                for the regularization parameter lmbda at iteration t

  """

  # Initialization 
  lmbda = []
  lmbda[0] = lmbda_max
  beta = dict()
  beta["lmbda0"] = 0

  for t in range(T-1):
    beta = beta["lambda" & (t-1)]

    for k in range(K):
      if k % f == 1:

        # Computation of theta 
        # Equation 11
        # Equation 18
        # Equation 19
        if G_lmbda(beta, theta) <= epsilon:
          beta_lmbda = beta
          break

        for j in A_C:
          beta_j = soft_thresholding(lmbda_t/np.linalg.norm(X[:,j])**2, beta_j - ((X[:,j].T @ (X @ beta - y))/np.linalg.norm(X[:,j])**2)

  return beta_lmbda_t